In [21]:
import duckdb

metadata_csv_path = '../task/data/metadata_category_clothing_shoes_and_jewelry_only.csv'
reviews_csv_path = '../task/data/reviews_Clothing_Shoes_and_Jewelry_5.csv'

# Connect DuckDB to an in-memory database
con = duckdb.connect(database=':memory:')

In [22]:
# exploring metadata csv
con.execute(f"CREATE VIEW metadata AS SELECT * FROM read_csv_auto('{metadata_csv_path}')")
df = con.execute("SELECT * FROM metadata LIMIT 5").df()

print(df)

   metadataid        asin           salesrank  \
0     2005401  B00004SR8Z  {'Clothing': 1631}   
1     2217897  B0000ZE74A  {'Clothing': 4742}   
2     2220611  B00012O12A     {'Clothing': 4}   
3     2324985  B0002XSXWC   {'Clothing': 300}   
4     2348742  B00066TWMU  {'Clothing': 1199}   

                                               imurl  \
0  http://ecx.images-amazon.com/images/I/41RfWLMD...   
1  http://ecx.images-amazon.com/images/I/41ryA-RO...   
2  http://ecx.images-amazon.com/images/I/41Dd7rCH...   
3  http://ecx.images-amazon.com/images/I/41-HS7D7...   
4  http://ecx.images-amazon.com/images/I/51UNLUB4...   

                                          categories  \
0  [['Clothing, Shoes & Jewelry', 'Luggage & Trav...   
1  [['Clothing, Shoes & Jewelry', 'Women', 'Cloth...   
2  [['Clothing, Shoes & Jewelry', 'Men', 'Clothin...   
3  [['Sports & Outdoors', 'Clothing', 'Men', 'Shi...   
4  [['Clothing, Shoes & Jewelry', 'ASICS'], ['Clo...   

                               

In [23]:
# exploring reviews csv
con.execute(f"CREATE VIEW reviews AS SELECT * FROM read_csv_auto('{reviews_csv_path}')")
df = con.execute("SELECT * FROM reviews LIMIT 5").df()

print(df)

   column0      reviewerID        asin                 reviewerName helpful  \
0        0  A1KLRMWW2FWPL4  0000031887  Amazon Customer "cameramom"  [0, 0]   
1        1  A2G5TCU2WDFZ65  0000031887              Amazon Customer  [0, 0]   
2        2  A1RLQXYNCMWRWN  0000031887                       Carola  [0, 0]   
3        3   A8U3FAMSJVHS5  0000031887                      Caromcg  [0, 0]   
4        4  A3GEOILWLK86XM  0000031887                           CJ  [0, 0]   

                                          reviewText  overall  \
0  This is a great tutu and at a really great pri...      5.0   
1  I bought this for my 4 yr old daughter for dan...      5.0   
2  What can I say... my daughters have it in oran...      5.0   
3  We bought several tutus at once, and they are ...      5.0   
4  Thank you Halo Heaven great product for Little...      5.0   

                         summary  unixReviewTime   reviewTime  
0  Great tutu-  not cheaply made      1297468800  02 12, 2011  
1     

In [24]:
# Analysis Example 1: Count the total number of reviews and the number of unique reviewers
query = """
SELECT
    COUNT(*) AS total_reviews
    , COUNT(DISTINCT reviewerID) AS unique_reviewers
FROM reviews
"""
review_counts = con.execute(query).df()
print(review_counts)

   total_reviews  unique_reviewers
0         278677             39387


In [25]:
# Analysis Example 2: Find top 5 reviewers by the number of reviews
query = """
SELECT
    reviewerID
    , COUNT(*) AS total_reviews
FROM reviews
GROUP BY reviewerID
ORDER BY total_reviews DESC
LIMIT 5
"""
top_reviewers = con.execute(query).df()
print(top_reviewers)

       reviewerID  total_reviews
0  A2J4XMWKR8PPD0            136
1  A2GA55P7WGHJCP             76
2  A2KBV88FL48CFS             69
3   AENH50GW3OKDA             68
4  A2V5R832QCSOMX             62


In [26]:
# Analysis Example 3: Get titles and prices of top 5 most reviewed products with price for 2014
query = """
SELECT 
    m.title
    , m.price
    , COUNT(*) AS total_reviews
FROM metadata AS m
LEFT JOIN reviews AS r ON m.asin = r.asin
WHERE TRUE
    AND m.price IS NOT NULL
    AND YEAR(strptime(r.reviewTime, '%m %d, %Y')) = 2014
GROUP BY ALL
ORDER BY total_reviews DESC
LIMIT 5
"""
top_products = con.execute(query).df()
print(top_products)

                                               title  price  total_reviews
0  Vintage, Retro Colorful Crystal Owl Pendant an...   3.00            172
1  Ladies Scoop Neck Stretchy Long Sleeve Knitted...   9.39             84
2  Sloggers  Women's Rain and Garden Boot with &q...  25.99             77
3   Lock Laces Elastic Shoelace and Fastening System   7.99             73
4  Retro Peacock Crystal Necklace Pendant Jewelry...   2.78             73


In [27]:
con.close()